In [1]:
from pstatsviewer.viewer import _calc_frames
from pstats import Stats
import pandas as pd

In [2]:
def enrich_stas_df(df):
    return pd.merge(
        (
            df.reset_index()["filename:funcname"]
                .str.split(":", n = 1, expand = True)
                .rename(columns={0:"file", 1: "function"})
                .assign(**{
                    "filename:funcname": df.reset_index()["filename:funcname"],
                    "module": [ 
                        row[-1:][0].split(":")[0].replace(".py", "")
                        for row in df.reset_index()["filename:funcname"].str.split("/")
                    ]
                })
            
        ),
        df.reset_index(),
        on="filename:funcname"
    ).drop(['filename:funcname'], axis=1)


In [9]:
generator_timings_df = enrich_stas_df(_calc_frames(Stats('../metrics/test_generator_consumption.prof'))[0])
iterator_timings_df = enrich_stas_df(_calc_frames(Stats('../metrics/test_iterator_consumption.prof'))[0])
chain_gen_timings_df = enrich_stas_df(_calc_frames(Stats('../metrics/test_chained_generator_consumption.prof'))[0])
chain_iter_timings_df = enrich_stas_df(_calc_frames(Stats('../metrics/test_chained_iterator_consumption.prof'))[0])

df = pd.concat([
    generator_timings_df.assign(type="generator"), 
    iterator_timings_df.assign(type="iterator"), 
    chain_gen_timings_df.assign(type="chained-generator"),
    chain_iter_timings_df.assign(type="chained-iterator")
]).sort_values(by=["function", "cumtime"])

In [10]:
df[df.function.str.startswith("test_consuming")].head(20)

,file,function,module,lineno,ccalls,ncalls,tottime,cumtime,type
8,/Users/oeid/Desktop/perfpy/test/performance/te...,test_consuming_attrs,test_generator_consumption,31,1,1,0.000001,16.730481,generator
8,/Users/oeid/Desktop/perfpy/test/performance/te...,test_consuming_attrs,test_chained_generator_consumption,38,1,1,0.000002,16.792903,chained-generator
11,/Users/oeid/Desktop/perfpy/test/performance/te...,test_consuming_attrs,test_chained_iterator_consumption,38,1,1,0.000001,17.371803,chained-iterator
11,/Users/oeid/Desktop/perfpy/test/performance/te...,test_consuming_attrs,test_iterator_consumption,31,1,1,0.000002,19.311175,iterator
9,/Users/oeid/Desktop/perfpy/test/performance/te...,test_consuming_dicts,test_generator_consumption,23,1,1,0.000002,13.377038,generator
9,/Users/oeid/Desktop/perfpy/test/performance/te...,test_consuming_dicts,test_chained_generator_consumption,30,1,1,0.000002,13.612898,chained-generator
12,/Users/oeid/Desktop/perfpy/test/performance/te...,test_consuming_dicts,test_chained_iterator_consumption,30,1,1,0.000001,13.628774,chained-iterator
12,/Users/oeid/Desktop/perfpy/test/performance/te...,test_consuming_dicts,test_iterator_consumption,23,1,1,0.000002,15.376774,iterator
10,/Users/oeid/Desktop/perfpy/test/performance/te...,test_consuming_named_tuples,test_generator_consumption,27,1,1,0.000001,14.843310,generator
10,/Users/oeid/Desktop/perfpy/test/performance/te...,test_consuming_named_tuples,test_chained_generator_consumption,34,1,1,0.000002,14.981407,chained-generator


In [6]:
df[(
    (df.type == "iterator") & 
    df.function.str.startswith("test_consuming")
)].sort_values(by=["module", "cumtime"])

,file,function,module,lineno,ccalls,ncalls,tottime,cumtime,type
12,/Users/oeid/Desktop/perfpy/test/performance/te...,test_consuming_dicts,test_iterator_consumption,23,1,1,0.000002,15.376774,iterator
13,/Users/oeid/Desktop/perfpy/test/performance/te...,test_consuming_named_tuples,test_iterator_consumption,27,1,1,0.000001,16.866209,iterator
11,/Users/oeid/Desktop/perfpy/test/performance/te...,test_consuming_attrs,test_iterator_consumption,31,1,1,0.000002,19.311175,iterator
14,/Users/oeid/Desktop/perfpy/test/performance/te...,test_consuming_pydantic_entities,test_iterator_consumption,19,1,1,0.000002,55.205076,iterator
